In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import *
from sklearn.multioutput import MultiOutputRegressor, RegressorChain

In [2]:
data = pd.read_csv('/workspaces/time-series/data/processed/time-serie.csv')
data.head()

,date,sales
0,2022-09-03,55.292157
1,2022-09-04,53.803211
2,2022-09-05,58.141693
3,2022-09-06,64.530899
4,2022-09-07,66.013633


In [3]:
data = data.set_index("date")  # establece la columna "date" como el índice del DataFrame
ts = data["sales"]
ts

date
2022-09-03      55.292157
2022-09-04      53.803211
2022-09-05      58.141693
2022-09-06      64.530899
2022-09-07      66.013633
                 ...     
2023-08-30     989.600354
2023-08-31     994.987326
2023-09-01     995.814415
2023-09-02     997.350214
2023-09-03    1000.482785
Name: sales, Length: 366, dtype: float64

In [4]:
data_train = ts.iloc[:-91]
data_test = ts.iloc[-91:]

In [5]:
data_train.tail()

date
2023-05-31    753.936865
2023-06-01    747.024687
2023-06-02    763.812942
2023-06-03    761.718225
2023-06-04    761.193459
Name: sales, dtype: float64

In [6]:
data_test.head()

date
2023-06-05    764.580565
2023-06-06    769.837390
2023-06-07    770.610592
2023-06-08    767.469590
2023-06-09    782.357862
Name: sales, dtype: float64

In [7]:
# Definir el tamaño de la ventana (n) para crear las secuencias de datos
n = 4

# Crear las secuencias de entrenamiento
XY_train = np.array([data_train.iloc[i:i+n].values for i in range(len(data_train)-n+1)])

# Extraer las características (X_train) y las etiquetas (y_train) de los datos de entrenamiento
X_train, y_train = XY_train[:, :-1], XY_train[:, -1]

# Crear las secuencias de prueba
XY_test = np.array([data_test.iloc[i:i+n].values for i in range(len(data_test)-n+1)])

# Extraer las características (X_test) y las etiquetas (y_test) de los datos de prueba
X_test, y_test = XY_test[:, :-1], XY_test[:, -1]

 - n = 4: Este es el tamaño de la ventana que determina cuántos puntos de datos consecutivos se utilizarán para predecir el siguiente punto de datos.
 - XY_train: Se crea una matriz que contiene secuencias de longitud n de datos de ventas consecutivos para el conjunto de entrenamiento. Se utiliza una lista de comprensión para recorrer todos los puntos de datos en el conjunto de entrenamiento y seleccionar secuencias de longitud n. Cada secuencia se convierte en un array NumPy y se agrega a XY_train.
 - X_train, y_train: Se separan las características (X_train) y las etiquetas (y_train) de las secuencias de entrenamiento XY_train. Las características (X_train) son todas las columnas excepto la última, que se usa como etiqueta (y_train).
 - XY_test: Se realiza un proceso similar para crear las secuencias de prueba, pero utilizando el conjunto de prueba en lugar del conjunto de entrenamiento.
 - X_test, y_test: Se separan las características (X_test) y las etiquetas (y_test) de las secuencias de prueba XY_test.

El objetivo de crear el array XY_train es estructurar los datos de entrenamiento de tal manera que puedan alimentarse fácilmente al modelo de regresión lineal. Cada fila de XY_train representa una secuencia de 4 puntos de datos de ventas consecutivos. Esto significa que cada fila contiene 4 valores de ventas consecutivos. Esto es útil porque en un modelo de regresión lineal, necesitamos características (X) y la variable objetivo (y). Al organizar los datos de esta manera, las características (X) serán los primeros 3 valores de ventas en cada fila, y la variable objetivo (y) será el último valor de ventas en cada fila.

In [8]:
# Crear el modelo de regresión lineal
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const)
# Ajustar el modelo a los datos
results = model.fit()

Añade una columna de unos (constante) a la matriz de características de entrenamiento X_train utilizando sm.add_constant().

Esta columna constante es necesaria para calcular el intercepto en el modelo de regresión lineal.

 En una regresión lineal, el término de intersección (o sesgo) es importante. Representa el valor esperado de la variable dependiente cuando todas las variables independientes son iguales a cero. Agregar una columna de unos a las características garantiza que el modelo incluya este término de intersección. Sin esta columna, el modelo asumiría que el término de intersección es cero, lo cual no sería apropiado en la mayoría de los casos.

Crea un modelo de regresión lineal ordinaria (OLS) utilizando la biblioteca statsmodels. Se ajusta el modelo utilizando la función sm.OLS(y_train, X_train_const), donde y_train es la variable de respuesta (la columna sales que se intenta predecir) y X_train_const es la matriz de características de entrenamiento, que incluye la columna constante añadida.

In [9]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.281e+05
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        15:44:28   Log-Likelihood:                -716.18
No. Observations:                 272   AIC:                             1440.
Df Residuals:                     268   BIC:                             1455.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3091      0.522     10.161      0.0

In [10]:
def wf_validation(X_train, y_train, X_test, y_test):
  y_pred = []
  for i in range(len(y_test)):
    yhat = LinearRegression().fit(X_train,y_train).predict(np.array([X_test[i]]))[0]
    y_pred.append(yhat)
    X_train, y_train = np.vstack((X_train, X_test[i])), np.hstack((y_train, y_test[i]))
  return y_pred
y_pred = wf_validation(X_train, y_train, X_test, y_test)

In [11]:
print(r2_score(y_test,y_pred), mean_absolute_percentage_error(y_test,y_pred)*100, mean_absolute_error(y_test,y_pred))

0.9974876317432907 0.302427674995026 2.6428857473343856


 El modelo tiene un R-cuadrado de aproximadamente 0.9975, lo que indica que explica la variabilidad en los datos de prueba muy bien.
 El error porcentual absoluto medio es del 0.302%, lo que sugiere que las predicciones del modelo están muy cerca de los valores reales. Además, el error absoluto medio es de aproximadamente 2.64, lo que indica que, en promedio, las predicciones del modelo están a una distancia de 2.64 unidades del valor real.
 
 En general, estos resultados sugieren que el modelo de regresión lineal tiene un buen rendimiento en la tarea de predicción.

In [15]:
in_len, out_len = 6, 2
n = in_len+out_len

XY_train = np.array([data_train[i:i+n].values for i in range(len(data_train)) if len(data_train[i:i+n])==n])
X_train, y_train = XY_train[:,:in_len], XY_train[:,-out_len:]

XY_test = np.array([data_test[i:i+n].values for i in range(len(data_test)) if len(data_test[i:i+n])==n])
X_test, y_test = XY_test[:,:in_len], XY_test[:,-out_len:]

In [16]:
data_train.values[:15]

array([55.29215704, 53.80321135, 58.1416934 , 64.53089878, 66.01363287,
       60.08186499, 68.46670361, 67.76510646, 70.51226125, 74.65645304,
       76.45952797, 82.99295751, 83.51598989, 84.20064149, 87.76994586])

In [17]:
X_train

array([[ 55.29215704,  53.80321135,  58.1416934 ,  64.53089878,
         66.01363287,  60.08186499],
       [ 53.80321135,  58.1416934 ,  64.53089878,  66.01363287,
         60.08186499,  68.46670361],
       [ 58.1416934 ,  64.53089878,  66.01363287,  60.08186499,
         68.46670361,  67.76510646],
       ...,
       [740.29236319, 743.90044019, 745.19677311, 746.60158806,
        750.4291868 , 753.93686506],
       [743.90044019, 745.19677311, 746.60158806, 750.4291868 ,
        753.93686506, 747.02468748],
       [745.19677311, 746.60158806, 750.4291868 , 753.93686506,
        747.02468748, 763.8129424 ]])

In [18]:
y_train

array([[ 68.46670361,  67.76510646],
       [ 67.76510646,  70.51226125],
       [ 70.51226125,  74.65645304],
       [ 74.65645304,  76.45952797],
       [ 76.45952797,  82.99295751],
       [ 82.99295751,  83.51598989],
       [ 83.51598989,  84.20064149],
       [ 84.20064149,  87.76994586],
       [ 87.76994586,  90.04211887],
       [ 90.04211887,  96.12607284],
       [ 96.12607284,  93.63110055],
       [ 93.63110055,  97.78851817],
       [ 97.78851817,  96.88976758],
       [ 96.88976758,  94.39582507],
       [ 94.39582507, 106.61839003],
       [106.61839003, 109.85358257],
       [109.85358257, 107.63651864],
       [107.63651864, 119.2750173 ],
       [119.2750173 , 110.70539613],
       [110.70539613, 117.80850843],
       [117.80850843, 119.71242105],
       [119.71242105, 127.47504997],
       [127.47504997, 129.88752836],
       [129.88752836, 128.54703406],
       [128.54703406, 131.81941907],
       [131.81941907, 130.62431399],
       [130.62431399, 129.94802155],
 

In [19]:
modelo = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)
y_pred_multi = modelo.predict(X_test)

In [20]:
print(r2_score(y_test[:,0],y_pred_multi[:,0]), mean_absolute_percentage_error(y_test[:,0],y_pred_multi[:,0])*100, mean_absolute_error(y_test[:,0],y_pred_multi[:,0]))

0.9975577434722676 0.28292286811871803 2.497648440853754


In [21]:
print(r2_score(y_test[:,1],y_pred_multi[:,1]), mean_absolute_percentage_error(y_test[:,1],y_pred_multi[:,1])*100, mean_absolute_error(y_test[:,1],y_pred_multi[:,1]))

0.9976857289401493 0.26755583506472796 2.369615605011944


In [22]:
model_chain = RegressorChain(LinearRegression()).fit(X_train, y_train)
y_pred_chain = model_chain.predict(X_test)

In [23]:
print(r2_score(y_test[:,0],y_pred_chain[:,0]), mean_absolute_percentage_error(y_test[:,0],y_pred_chain[:,0])*100, mean_absolute_error(y_test[:,0],y_pred_chain[:,0]))

0.9975577434722676 0.28292286811871803 2.497648440853754


In [24]:
print(r2_score(y_test[:,1],y_pred_chain[:,1]), mean_absolute_percentage_error(y_test[:,1],y_pred_chain[:,1])*100, mean_absolute_error(y_test[:,1],y_pred_chain[:,1]))

0.9976857289401493 0.26755583506472846 2.369615605011948
